In [3]:

############################### INITIALISATION ##########################################
# GENERAL LIBRARIES
import json
import os
import sys
import contextlib
import io
import numpy as np
import inspect
import os.path
import matplotlib.pyplot as plt
from math import pi, atan, sin, cos, sqrt, tanh, cosh, exp, ceil
import seaborn as sns
from numpy.core.fromnumeric import shape
import random
import time
import math
from tqdm import tqdm
from pathlib import Path


########################## LOCAL DEPENDENCIES #################################

filename = inspect.getframeinfo(inspect.currentframe()).filename
file_path = os.path.dirname(os.path.abspath(filename))

#project_root = os.path.dirname(file_path)
project_root = file_path

if project_root not in sys.path:
    sys.path.append(project_root)

# PROJECT MODULES
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
project_root = module_path

print(project_root)

from event_model import event_model as em
from validator import validator_lite as vl
import data_analysis.event_generator as eg
from visual.color_map import Colormap


c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub


In [4]:
def load_event(file_name, plot_event=False):
    f = open(file_name)
    json_data_event = json.loads(f.read())

    ev = em.event(json_data_event, read_tracks=True)

    modules = ev.modules
    tracks = ev.real_tracks

    if plot_event:
        eg.plot_tracks_and_modules(tracks, modules, title="Loaded Event")

    modules_even = []
    modules_odd = []

    for i in range(len(modules)):
        if i % 2 == 0:
            modules_even.append(modules[i])
        else:
            modules_odd.append(modules[i])

    return json_data_event, (modules_even, modules_odd), modules

In [5]:
def evaluate_events_1(file_name, parameters, nr_events=1, plot_event=False, output_file=None):

    json_data_all_events = []
    all_tracks = []

    nr_max_neurons_tracking = []
    id_events_tracking = []
    total_hits_tracking = []
    timing_tracking = []
    start_time_networks = time.time() 

    all_events = [i for i in range(995)]
    random.seed(40)
    random.shuffle(all_events)
    count = 0
    j = 0
    
    while count < nr_events:
        i = all_events[j]
        j += 1

        try:
            size = os.path.getsize(file_name + str(i) + ".json")
            print("[INFO] Evaluate Event: %s" % file_name + str(i))
            start_timing = time.time() 
            json_data_event, (modules_even, modules_odd), modules = load_event(
                file_name + str(i) + ".json", plot_event=False
            )
            
            print(modules)

            total_hits = 0
            max_neurons = 0
            last = 0

            for m in modules:
                n_hits = len(m.hits())
                if last * n_hits > max_neurons:
                    max_neurons = last * n_hits
                last = n_hits
                total_hits = total_hits + n_hits
            
            nr_max_neurons_tracking.append(max_neurons)
            total_hits_tracking.append(total_hits)
            id_events_tracking.append(i)

            print(f"\nstarting instance {count+1} out of {nr_events}\n")
            print(f'Number of total hits: {total_hits}')
            print(f'Number of max neurons: {max_neurons}')
            count +=1
        except:
            continue

In [6]:
def evaluate_events(file_name, parameters, nr_events=1, plot_event=False, output_file=None):

    json_data_all_events = []
    all_tracks = []
    iter_even = 1
    iter_odd = 1

    nr_max_neurons_tracking = []
    id_events_tracking = []
    total_hits_tracking = []
    timing_tracking = []
    start_time_networks = time.time() 

    all_events = [i for i in range(995)]
    random.seed(40)
    random.shuffle(all_events)
    count = 0
    j = 0
    
    while count < nr_events:
        i = all_events[j]
        j += 1

        try:
            size = os.path.getsize(file_name + str(i) + ".json")
            print("[INFO] Evaluate Event: %s" % file_name + str(i))
            start_timing = time.time() 
            json_data_event, modules = load_event(
                file_name + str(i) + ".json", plot_event=False
            )
            
            total_hits = 0
            max_neurons = 0
            last = 0

            for m in modules[0]:
                n_hits = len(m.hits())
                if last * n_hits > max_neurons:
                    max_neurons = last * n_hits
                last = n_hits
                total_hits = total_hits + n_hits
            

            last = 0
            for m in modules[1]:
                n_hits = len(m.hits())
                if last * n_hits > max_neurons:
                    max_neurons = last * n_hits
                last = n_hits
                total_hits = total_hits + n_hits
            
            nr_max_neurons_tracking.append(max_neurons)
            total_hits_tracking.append(total_hits)
            id_events_tracking.append(i)

            print(f"\nstarting instance {count+1} out of {nr_events}\n")
            print(f'Number of total hits: {total_hits}')
            print(f'Number of max neurons: {max_neurons}')
            count += 1
            print(count)

        except:
            continue
        

In [7]:
file_name =  project_root + f"/datasets/minibias/velo_event_"
parameters = {
        ### NEURONS ###
        "random_neuron_init": True,
        "binary_states": False,  # try it out once maybe but scrap it
        ### WEIGHTS ###
        "ALPHA": 1,
        "BETA": 10,
        "GAMMA": 10,
        "narrowness": 200,
        "constant_factor": 0.9,
        "monotone_constant_factor": 0.9,
        #### UPDATE ###
        "T": 1e-8,  # try to experiment with these rather
        "B": 1e-6,  # try to experiment with these rather
        "T_decay": lambda t: max(1e-8, t * 0.01),  # try to remove these
        "B_decay": lambda t: max(1e-4, t * 0.04),  # try to remove these
        "decay_off": False,  # using this
        "randomized_updates": True,
        "fully_randomized_updates": False,
        #### THRESHOLD ###
        "maxActivation": True,
        "THRESHOLD": 0.2,
        ##### CONVERGENCE ###
        "convergence_threshold": 0.00000005,
        "bootstrap_iters": 10,
        "bootstrap_method": "below_mean",
        ###### BIFURC REMOVAL #####
        "smart": True,
        "only_weight": False,
        "max_activation": False,
        ###### Track prunning #######
        # here we could set the threshold
        "pruning_tr": 0.05,
    }
print(file_name)

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub/datasets/minibias/velo_event_


In [8]:
evaluate_events_1(file_name, parameters, nr_events=1, plot_event=False, output_file=None)


[INFO] Evaluate Event: c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub/datasets/minibias/velo_event_839
[module 0:
 At z: {-288.08, -286.918}
 Number of hits: 16
 Hits (#id {x, y, z}): [#0 module 0 {9.297, -16.897, -288.08}, #1 module 0 {-6.531, -2.313, -288.08}, #2 module 0 {5.246, -13.144, -288.08}, #3 module 0 {-18.658, -14.182, -286.918}, #4 module 0 {-12.494, -21.902, -286.918}, #5 module 0 {-13.446, -24.643, -286.918}, #6 module 0 {-14.263, -30.205, -286.918}, #7 module 0 {-8.138, -25.752, -286.918}, #8 module 0 {-1.546, -43.194, -286.918}, #9 module 0 {-6.708, 7.932, -286.918}, #10 module 0 {-13.63, 1.943, -286.918}, #11 module 0 {-16.022, -5.348, -286.918}, #12 module 0 {-20.494, -11.61, -286.918}, #13 module 0 {-34.573, -6.243, -288.08}, #14 module 0 {-18.258, 15.166, -288.08}, #15 module 0 {-23.256, 11.841, -288.08}], module 1:
 At z: {-275.58, -274.418}
 Number of hits: 10
 Hits (#id {x, y, z}): [#16 module 1 {1.262, 8.789, -275.58}, #17 module 1 {4.918